In [1]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import seaborn as sns
import pandas as pd
import scanpy as sc
import anndata as ad
from sklearn.cluster import AgglomerativeClustering, KMeans
from sklearn.metrics import adjusted_rand_score
import community
import networkx as nx
import gzip
from scipy.sparse import csc_matrix, csr_matrix
import scipy.sparse
from scipy.io import mmwrite
import scib
from scipy import stats

import anndata2ri
import logging

import rpy2.rinterface_lib.callbacks as rcb
import rpy2.robjects as ro

sc.settings.verbosity = 0
sc.settings.set_figure_params(
    dpi=80,
    facecolor="white",
    frameon=False,
)

rcb.logger.setLevel(logging.ERROR)
ro.pandas2ri.activate()
anndata2ri.activate()

%load_ext rpy2.ipython

%matplotlib inline

/home/ubuntu/myvenv/lib/python3.10/site-packages/scanpy/_settings.py:447: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  IPython.display.set_matplotlib_formats(*ipython_format)
/home/ubuntu/myvenv/lib/python3.10/site-packages/rpy2/robjects/pandas2ri.py:368: DeprecationWarning: The global conversion available with activate() is deprecated and will be removed in the next major release. Use a local converter.
  warnings.warn('The global conversion available with activate() '
/home/ubuntu/myvenv/lib/python3.10/site-packages/rpy2/robjects/numpy2ri.py:241: DeprecationWarning: The global conversion available with activate() is deprecated and will be removed in the next major release. Use a local converter.
  warnings.warn('The global conversion available with activate() '
/tmp/ipykernel_130892/2519990529.py:35: DeprecationWarning: The global conversion available with activate() is deprec

In [2]:

###############
dataset="human_pancreas_norm_complexBatch"
###############

data_dir="/goofys/users/Aleksandra_S/benchmarking_datasets/"+dataset+".h5ad"
adata=ad.read_h5ad(data_dir)

/home/ubuntu/myvenv/lib/python3.10/site-packages/anndata/_io/specs/registry.py:235: OldFormatWarning: Element '/X' was written without encoding metadata.
  return self.callback(read_func, elem.name, elem, iospec=get_spec(elem))
/home/ubuntu/myvenv/lib/python3.10/site-packages/anndata/_io/specs/registry.py:235: OldFormatWarning: Element '/layers' was written without encoding metadata.
  return self.callback(read_func, elem.name, elem, iospec=get_spec(elem))
/home/ubuntu/myvenv/lib/python3.10/site-packages/anndata/_io/specs/registry.py:235: OldFormatWarning: Element '/layers/counts' was written without encoding metadata.
  return self.callback(read_func, elem.name, elem, iospec=get_spec(elem))
/home/ubuntu/myvenv/lib/python3.10/site-packages/anndata/_io/utils.py:202: OldFormatWarning: Element '/obs/__categories/tech' was written without encoding metadata.
  return func(*args, **kwargs)
/home/ubuntu/myvenv/lib/python3.10/site-packages/anndata/_io/utils.py:202: OldFormatWarning: Element '/

In [5]:
adata.obs['batch'] = adata.obs.tech

In [7]:
unique_batches = adata.obs['batch'].unique()
# unpack anndata (divide into batches)
# Initialize a dictionary to store the split AnnData objects
split_adatas = {}

# Split the original AnnData object into separate batches
for batch in unique_batches:
    mask = adata_raw.obs['batch'] == batch
    split_adatas[batch] = adata_raw[mask].copy()
adata_list =  [adata for adata in split_adatas.values()]
#split_adatas

In [8]:
unique_batches

['celseq', 'celseq2', 'fluidigmc1', 'smartseq2', 'inDrop1', 'inDrop2', 'inDrop3', 'inDrop4', 'smarter']
Categories (9, object): ['celseq', 'celseq2', 'fluidigmc1', 'inDrop1', ..., 'inDrop3', 'inDrop4', 'smarter', 'smartseq2']

In [101]:
list_index = 8 # if == "All", it means there will be no slicing by cluster
slice_name = 'c5'
if list_index == 'All':
    batch_name = ""
else:
    batch_name = unique_batches[list_index]

### Slicing Cluster (optional)

In [102]:
if list_index != 'All':
    adata = adata_list[list_index].copy()

## Highly Deviant Genes
racuna se nad RAW podacima! <br>
quantifies whether genes show a constant expression profile across cells as these are not informative.  <br>
Genes with constant expression are described by a multinomial null model, they are approximated by the binomial deviance.  <br>
Highly informative genes across cells will have a high deviance value which indicates a poor fit by According to the deviance values, the method then ranks all genes and obtains only highly deviant genes. <br>

In [103]:
%%R
library(scry)

In [104]:
raw_adata = ad.AnnData(X=adata.layers['counts'])

In [105]:
ro.globalenv["adata"] = raw_adata

In [ ]:
%%R
sce = devianceFeatureSelection(adata, assay="X")

In [ ]:
binomial_deviance = ro.r("rowData(sce)$binomial_deviance").T

In [ ]:
idx = binomial_deviance.argsort()[-5000:]
mask = np.zeros(adata.var_names.shape, dtype=bool)
mask[idx] = True

adata.var["highly_deviant"] = mask
adata.var["binomial_deviance"] = binomial_deviance

In [ ]:
highly_deviant_df = adata.var[adata.var.highly_deviant].sort_values(by='binomial_deviance', ascending=False)
highly_deviant_names = highly_deviant_df.index.values

In [ ]:
highly_deviant_df.to_csv(f"/goofys/users/Aleksandra_S/benchmarking_datasets/highly_deviant/highly_deviant_{batch_name}.csv")